# 4.1 판다스 패키지의 소개

대부분의 데이터는 시계열(series)이나 표(table)의 형태로 나타낼 수 있다. 판다스(Pandas) 패키지는 이러한 데이터를 다루기 위한 시리즈(Series) 클래스와 데이터프레임(DataFrame) 클래스를 제공한다.

- 시리즈: 열
- 데이터프레임: 2차원 형식으로 이루어진 표

In [1]:
import numpy as np
import pandas as pd

### 넘파이 vs 판다스 인덱싱

```
이름  | numpy | pandas
------|-------|--------
홍길동|  [0]  | first
------|-------|--------
대조영|  [1]  | second
```

## 시리즈 클래스

### 시리즈 생성

In [2]:
s = pd.Series([9904312, 3448737, 2890451, 2466052],
              index=["서울", "부산", "인천", "대구"])
s

서울    9904312
부산    3448737
인천    2890451
대구    2466052
dtype: int64

In [3]:
x = np.array([9904312, 3448737, 2890451, 2466052])
x

array([9904312, 3448737, 2890451, 2466052])

In [4]:
pd.Series(range(10, 13))

0    10
1    11
2    12
dtype: int64

In [5]:
s.index

Index(['서울', '부산', '인천', '대구'], dtype='object')

In [6]:
s.values

array([9904312, 3448737, 2890451, 2466052], dtype=int64)

In [7]:
s.name = "인구"
s.index.name = "도시"
s

도시
서울    9904312
부산    3448737
인천    2890451
대구    2466052
Name: 인구, dtype: int64

### 시리즈 연산

In [8]:
s / 1000000

도시
서울    9.904312
부산    3.448737
인천    2.890451
대구    2.466052
Name: 인구, dtype: float64

### 시리즈 인덱싱

In [9]:
s[1], s["부산"]

(3448737, 3448737)

In [10]:
s[3], s["대구"]

(2466052, 2466052)

배열 인덱싱을 하면 부분적인 값을 가지는 시리즈 자료형을 반환한다. 자료의 순서를 바꾸거나 특정한 자료만 선택할 수 있다.

In [11]:
s[[0, 3, 1]]

도시
서울    9904312
대구    2466052
부산    3448737
Name: 인구, dtype: int64

In [12]:
s[["서울", "대구", "부산"]]

도시
서울    9904312
대구    2466052
부산    3448737
Name: 인구, dtype: int64

In [13]:
# 인구가 250만 초과, 500만 미만인 경우
# 1만 = 10000 = e4
s[(250e4 < s) & (s < 400e4)]

도시
부산    3448737
인천    2890451
Name: 인구, dtype: int64

In [14]:
# 두번째(1)부터 세번째(2)까지 (네번째(3) 미포함)
s[1:3]

도시
부산    3448737
인천    2890451
Name: 인구, dtype: int64

In [15]:
# 부산에서 대구까지 (대구도 포함)
s["부산":"대구"]

도시
부산    3448737
인천    2890451
대구    2466052
Name: 인구, dtype: int64

In [16]:
# 만약 라벨 값이 영문 문자열인 경우에는 인덱스 라벨이 속성인것처럼 점(.)을 이용하여 해당 인덱스 값에 접근할 수도 있다.
s0 = pd.Series(range(3), index=["a", "b", "c"])
s0.a, s0.b, s0.c

(0, 1, 2)

### 시리즈와 딕셔너리 자료형

시리즈 객체는 라벨 값에 의해 인덱싱이 가능하므로 실질적으로 인덱스 라벨 값을 키(key)로 가지는 딕셔너리 자료형과 같다고 볼 수 있다. 따라서 딕셔너리 자료형에서 제공하는 in 연산도 가능하고 items 메서드를 사용하면 for 루프를 통해 각 원소의 키(key)와 값(value)을 접근할 수도 있다.

In [17]:
# 인덱스 라벨 중에 서울이 있는가
"서울" in s

True

In [18]:
# 인덱스 라벨 중에 대전이 있는가
"대전" in s

False

In [19]:
for k, v in s.items():
    print("%s = %d" % (k, v))

서울 = 9904312
부산 = 3448737
인천 = 2890451
대구 = 2466052


In [20]:
for k, v in s.items():
    print(f"{k} = {v}")

서울 = 9904312
부산 = 3448737
인천 = 2890451
대구 = 2466052


In [21]:
s2 = pd.Series({"서울": 9631482, "부산": 3393191, "인천": 2632035, "대전": 1490158})
s2

서울    9631482
부산    3393191
인천    2632035
대전    1490158
dtype: int64

딕셔너리의 원소는 순서를 가지지 않으므로 시리즈의 데이터도 순서가 보장되지 않는다. 만약 순서를 정하고 싶다면 인덱스를 리스트로 지정해야 한다.

In [22]:
s2 = pd.Series({"서울": 9631482, "부산": 3393191, "인천": 2632035, "대전": 1490158},
               index=["부산", "서울", "인천", "대전"])
s2

부산    3393191
서울    9631482
인천    2632035
대전    1490158
dtype: int64

### 인덱스 기반 연산

이번에는 2015년도와 2010년의 인구 증가를 계산해 보자.

대구와 대전의 경우에는 2010년 자료와 2015년 자료가 모두 존재하지 않기 때문에 계산이 불가능하므로 NaN(Not a Number)이라는 값을 가지게 된다. 또한 NaN 값이 float 자료형에서만 가능하므로 다른 계산 결과도 모두 float 자료형이 되었다는 점에 주의한다. 

In [23]:
ds = s - s2
ds

대구         NaN
대전         NaN
부산     55546.0
서울    272830.0
인천    258416.0
dtype: float64

In [24]:
s.values - s2.values

array([ 6511121, -6182745,   258416,   975894], dtype=int64)

NaN이 아닌 값을 구하려면 notnull 메서드를 사용한다.

In [25]:
ds.notnull()

대구    False
대전    False
부산     True
서울     True
인천     True
dtype: bool

In [26]:
ds[ds.notnull()]

부산     55546.0
서울    272830.0
인천    258416.0
dtype: float64

마찬가지로 인구 증가율(%)은 다음과 같이 구할 수 있다.

In [27]:
rs = (s - s2) / s2 * 100
rs = rs[rs.notnull()]
rs

부산    1.636984
서울    2.832690
인천    9.818107
dtype: float64

### 데이터의 갱신, 추가, 삭제

In [28]:
rs["부산"] = 1.63
rs

부산    1.630000
서울    2.832690
인천    9.818107
dtype: float64

In [29]:
rs["대구"] = 1.41
rs

부산    1.630000
서울    2.832690
인천    9.818107
대구    1.410000
dtype: float64

In [30]:
del rs["서울"]
rs

부산    1.630000
인천    9.818107
대구    1.410000
dtype: float64

### 연습 문제 4.1.1

- 임의로 두 개의 시리즈 객체를 만든다. 모두 문자열 인덱스를 가져야 하며 두 시리즈에 공통적으로 포함되지 않는 라벨이 있어야 한다.
- 위에서 만든 두 시리즈 객체를 이용하여 사칙 연산을 한다.

In [31]:
d1 = pd.Series({"삼성":10000, "엘지":11000, "롯데":12000, "현대":13000, "기아":5000})
d2 = pd.Series({"삼성":10000, "엘지":12000, "롯데":11000, "현대":9000, "기아":8000, "대우":7000})

In [32]:
# 주가 등락률
d3 = (d2 - d1) / d1 * 100
d3 = d3[d3.notnull()]
d3

기아    60.000000
롯데    -8.333333
삼성     0.000000
엘지     9.090909
현대   -30.769231
dtype: float64

In [33]:
r1 = np.random.randint(5000, 10000, 30)
print(r1)

[9451 9084 7090 7227 9439 9870 9794 7442 7506 5239 8278 7889 8828 9805
 8047 9066 7233 5765 8194 9035 7681 9597 9803 7319 9265 9002 5840 5708
 9281 9809]


In [34]:
r1.max(), r1.min(), r1.mean()

(9870, 5239, 8252.9)

In [35]:
r1.std(), r1.var(), np.median(r1)

(1359.1583265634165, 1847311.3566666665, 8553.0)

In [36]:
np.percentile(r1, 0), np.percentile(r1, 25), np.percentile(r1, 50), np.percentile(r1, 75), np.percentile(r1, 100)

(5239.0, 7349.75, 8553.0, 9399.5, 9870.0)

## 데이터프레임 클래스

시리즈가 1차원 벡터 데이터에 행방향 인덱스(row index)를 붙인 것이라면 데이터프레임 DataFrame 클래스는 2차원 행렬 데이터에 인덱스를 붙인 것과 비슷하다. 2차원이므로 각각의 행 데이터의 이름이 되는 행 인덱스(row index) 뿐 아니라 각각의 열 데이터의 이름이 되는 열 인덱스(column index)도 붙일 수 있다.

### 데이터프레임 생성

앞에서 데이터프레임은 2차원 배열 데이터를 기반으로 한다고 했지만 사실은 공통 인덱스를 가지는 열 시리즈(column series)를 딕셔너리로 묶어놓은 것이라고 보는 것이 더 정확하다.

In [37]:
data = {
    "2015": [9904312, 3448737, 2890451, 2466052],
    "2010": [9631482, 3393191, 2632035, 2431774],
    "2005": [9762546, 3512547, 2517680, 2456016],
    "2000": [9853972, 3655437, 2466338, 2473990],
    "지역": ["수도권", "경상권", "수도권", "경상권"],
    "2010-2015 증가율": [0.0283, 0.0163, 0.0982, 0.0141]
}
columns = ["지역", "2015", "2010", "2005", "2000", "2010-2015 증가율"]
index = ["서울", "부산", "인천", "대구"]
df = pd.DataFrame(data, index=index, columns=columns)
df

,지역,2015,2010,2005,2000,2010-2015 증가율
서울,수도권,9904312,9631482,9762546,9853972,0.0283
부산,경상권,3448737,3393191,3512547,3655437,0.0163
인천,수도권,2890451,2632035,2517680,2466338,0.0982
대구,경상권,2466052,2431774,2456016,2473990,0.0141


In [38]:
data = {
    "2015": [9904312, 3448737, 2890451, 2466052],
    "2010": [9631482, 3393191, 2632035, 2431774],
    "2005": [9762546, 3512547, 2517680, 2456016],
    "2000": [9853972, 3655437, 2466338, 2473990],
    "지역": ["수도권", "경상권", "수도권", "경상권"],
    "2010-2015 증가율": [0.0283, 0.0163, 0.0982, 0.0141]
}
index = ["서울", "부산", "인천", "대구"]
df = pd.DataFrame(data, index=index)
df

,2015,2010,2005,2000,지역,2010-2015 증가율
서울,9904312,9631482,9762546,9853972,수도권,0.0283
부산,3448737,3393191,3512547,3655437,경상권,0.0163
인천,2890451,2632035,2517680,2466338,수도권,0.0982
대구,2466052,2431774,2456016,2473990,경상권,0.0141


In [39]:
df.values

array([[9904312, 9631482, 9762546, 9853972, '수도권', 0.0283],
       [3448737, 3393191, 3512547, 3655437, '경상권', 0.0163],
       [2890451, 2632035, 2517680, 2466338, '수도권', 0.0982],
       [2466052, 2431774, 2456016, 2473990, '경상권', 0.0141]], dtype=object)

In [40]:
df.columns

Index(['2015', '2010', '2005', '2000', '지역', '2010-2015 증가율'], dtype='object')

In [41]:
df.index

Index(['서울', '부산', '인천', '대구'], dtype='object')

### 연습 문제 4.1.2

다음 조건을 만족하는 임의의 데이터프레임을 하나 만든다.

- 열의 갯수와 행의 갯수가 각각 5개 이상이어야 한다.
- 열에는 정수, 문자열, 실수 자료형 데이터가 각각 1개 이상씩 포함되어 있어야 한다.

In [42]:
r1 = np.random.randint(5000, 10000, 5)
r1

array([6487, 6927, 5871, 5263, 8219])

In [43]:
data = {}
for i in range(0, 11):
    print(i)
    data[i]

# list(r1)
data

0


KeyError: 0

### 열 데이터의 갱신, 추가, 삭제

In [151]:
# "2010-2015 증가율"이라는 이름의 열 추가
df["2010-2015 증가율"] = df["2010-2015 증가율"] * 100
df

,2015,2010,2005,2000,지역,2010-2015 증가율
서울,9904312,9631482,9762546,9853972,수도권,2.83
부산,3448737,3393191,3512547,3655437,경상권,1.63
인천,2890451,2632035,2517680,2466338,수도권,9.82
대구,2466052,2431774,2456016,2473990,경상권,1.41


In [152]:
# "2005-2010 증가율"이라는 이름의 열 추가
df["2005-2010 증가율"] = ((df["2010"] - df["2005"]) / df["2005"] * 100).round(2)
df

,2015,2010,2005,2000,지역,2010-2015 증가율,2005-2010 증가율
서울,9904312,9631482,9762546,9853972,수도권,2.83,-1.34
부산,3448737,3393191,3512547,3655437,경상권,1.63,-3.40
인천,2890451,2632035,2517680,2466338,수도권,9.82,4.54
대구,2466052,2431774,2456016,2473990,경상권,1.41,-0.99


In [153]:
# "2010-2015 증가율"이라는 이름의 열 삭제
del df["2010-2015 증가율"]
df

,2015,2010,2005,2000,지역,2005-2010 증가율
서울,9904312,9631482,9762546,9853972,수도권,-1.34
부산,3448737,3393191,3512547,3655437,경상권,-3.40
인천,2890451,2632035,2517680,2466338,수도권,4.54
대구,2466052,2431774,2456016,2473990,경상권,-0.99


## 열 인덱싱

In [156]:
# 하나의 열만 인덱싱하면 시리즈가 반환된다.
df["지역"]

서울    수도권
부산    경상권
인천    수도권
대구    경상권
Name: 지역, dtype: object

In [157]:
# 여러개의 열을 인덱싱하면 부분적인 데이터프레임이 반환된다.
df[["2010", "2015"]]

,2010,2015
서울,9631482,9904312
부산,3393191,3448737
인천,2632035,2890451
대구,2431774,2466052


In [158]:
# 2010이라는 열을 반환하면서 데이터프레임 자료형을 유지
df[["2010"]]

,2010
서울,9631482
부산,3393191
인천,2632035
대구,2431774


In [161]:
type(df["2010"])

pandas.core.series.Series

In [163]:
type(df[["2010"]])

pandas.core.frame.DataFrame

데이터프레임의 열 인덱스가 문자열 라벨을 가지고 있는 경우에는 순서를 나타내는 정수 인덱스를 열 인덱싱에 사용할 수 없다.

In [169]:
df[0]

KeyError: 0

In [165]:
df['2000']

서울    9853972
부산    3655437
인천    2466338
대구    2473990
Name: 2000, dtype: int64

다만 원래부터 문자열이 아닌 정수형 열 인덱스를 가지는 경우에는 인덱스 값으로 정수를 사용할 수 있다.

In [166]:
df2 = pd.DataFrame(np.arange(12).reshape(3, 4))
df2

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [168]:
df2[0]

0    0
1    4
2    8
Name: 0, dtype: int32

## 행 인덱싱

만약 행 단위로 인덱싱을 하고자 하면 항상 슬라이싱(slicing)을 해야 한다. 인덱스의 값이 문자 라벨이면 라벨 슬라이싱도 가능하다.

In [171]:
df[:1]

,2015,2010,2005,2000,지역,2005-2010 증가율
서울,9904312,9631482,9762546,9853972,수도권,-1.34


In [181]:
df[:"인천"]

,2015,2010,2005,2000,지역,2005-2010 증가율
서울,9904312,9631482,9762546,9853972,수도권,-1.34
부산,3448737,3393191,3512547,3655437,경상권,-3.40
인천,2890451,2632035,2517680,2466338,수도권,4.54


In [172]:
df[1:2]

,2015,2010,2005,2000,지역,2005-2010 증가율
부산,3448737,3393191,3512547,3655437,경상권,-3.4


In [173]:
df[1:3]

,2015,2010,2005,2000,지역,2005-2010 증가율
부산,3448737,3393191,3512547,3655437,경상권,-3.40
인천,2890451,2632035,2517680,2466338,수도권,4.54


In [174]:
df["서울":"부산"]

,2015,2010,2005,2000,지역,2005-2010 증가율
서울,9904312,9631482,9762546,9853972,수도권,-1.34
부산,3448737,3393191,3512547,3655437,경상권,-3.40


## 개별 데이터 인덱싱

데이터프레임에서 열 라벨로 시리즈를 인덱싱하면 시리즈가 된다. 이 시리즈를 다시 행 라벨로 인덱싱하면 개별 데이터가 나온다.

In [177]:
# 데이터프레임[열][행]
df["2015"]["서울"]

9904312

### 연습 문제 4.1.3

다음 데이터프레임에서 지정하는 데이터를 뽑아내거나 처리하라.
```
data = {
    "국어": [80, 90, 70, 30],
    "영어": [90, 70, 60, 40],
    "수학": [90, 60, 80, 70],
}
columns = ["국어", "영어", "수학"]
index = ["춘향", "몽룡", "향단", "방자"]
df = pd.DataFrame(data, index=index, columns=columns)
```

In [45]:
data = {
    "국어": [80, 90, 70, 30],
    "영어": [90, 70, 60, 40],
    "수학": [90, 60, 80, 70],
}
columns = ["국어", "영어", "수학"]
index = ["춘향", "몽룡", "향단", "방자"]
df = pd.DataFrame(data, index=index, columns=columns)
df

,국어,영어,수학
춘향,80,90,90
몽룡,90,70,60
향단,70,60,80
방자,30,40,70


In [46]:
# (1) 모든 학생의 수학 점수를 시리즈로 나타낸다.
df["수학"]

춘향    90
몽룡    60
향단    80
방자    70
Name: 수학, dtype: int64

In [47]:
# (2) 모든 학생의 국어와 영어 점수를 데이터 프레임으로 나타낸다.
df[["국어", "영어"]]

,국어,영어
춘향,80,90
몽룡,90,70
향단,70,60
방자,30,40


In [50]:
# (3) 모든 학생의 각 과목 평균 점수를 새로운 열로 추가한다.
df["평균"] = ((df["국어"] + df["영어"] + df["수학"]) / 3).round(2)
df

,국어,영어,수학,평균
춘향,80,90,90,86.67
몽룡,90,70,60,73.33
향단,70,60,80,70.00
방자,30,40,70,46.67


In [51]:
df["평균"] = df.mean(axis=1).round(2)
df

,국어,영어,수학,평균
춘향,80,90,90,86.67
몽룡,90,70,60,73.33
향단,70,60,80,70.00
방자,30,40,70,46.67


In [52]:
# (4) 방자의 영어 점수를 80점으로 수정하고 평균 점수도 다시 계산한다.
df["영어"]["방자"] = 80
df["평균"] = ((df["국어"] + df["영어"] + df["수학"]) / 3).round(2)
df

,국어,영어,수학,평균
춘향,80,90,90,86.67
몽룡,90,70,60,73.33
향단,70,60,80,70.00
방자,30,80,70,60.00


In [53]:
df["영어"]["방자"] = 80
df["평균"] = df.mean(axis=1).round(2)
df

,국어,영어,수학,평균
춘향,80,90,90,86.67
몽룡,90,70,60,73.33
향단,70,60,80,70.00
방자,30,80,70,60.00


In [54]:
# (5) 춘향의 점수를 데이터프레임으로 나타낸다.
df["춘향":"춘향"]

,국어,영어,수학,평균
춘향,80,90,90,86.67


In [55]:
df["춘향":"춘향"].T

,춘향
국어,80.00
영어,90.00
수학,90.00
평균,86.67


In [203]:
type(df["춘향":"춘향"])

pandas.core.frame.DataFrame

In [58]:
# (6) 향단의 점수를 시리즈로 나타낸다.
df.T["향단"]

국어    70.0
영어    60.0
수학    80.0
평균    70.0
Name: 향단, dtype: float64

In [56]:
df.T

,춘향,몽룡,향단,방자
국어,80.00,90.00,70.0,30.0
영어,90.00,70.00,60.0,80.0
수학,90.00,60.00,80.0,70.0
평균,86.67,73.33,70.0,60.0
